In [ ]:
import micromegas
import datetime
import pandas as pd
pd.set_option('display.max_colwidth', None)
client = micromegas.connect()

In [ ]:
micromegas.admin.list_incompatible_partitions(client)

In [ ]:
%%time
sql = """
SELECT view_set_name, view_instance_id, min(begin_insert_time) as begin, max(end_insert_time) as end
FROM list_partitions()
where view_instance_id != 'global'
group by view_set_name, view_instance_id
"""
df_instances = client.query(sql)
for _, row in df_instances.iterrows():
    client.retire_partitions(row["view_set_name"], row["view_instance_id"], row["begin"], row["end"])

In [ ]:
sql = "SELECT * FROM list_partitions() limit 2"
df = client.query(sql)
display(sql)
df

In [ ]:
%%time
sql = """
SELECT block_id, begin_time, end_time, property_get("streams.properties", 'thread-name') as thread_name, property_get("streams.properties", 'thread-id') as thread_id
FROM blocks
WHERE process_id = '5589319b-83a3-4d48-9485-5926453b35d1'
AND ARRAY_HAS(blocks."streams.tags", 'cpu')
ORDER BY block_id
"""
client.query(sql)

In [ ]:
%%time
now = datetime.datetime.now(datetime.timezone.utc)
begin = now - datetime.timedelta(days=10)
begin = datetime.datetime.combine(begin, datetime.time(), tzinfo=begin.tzinfo)
end = datetime.datetime.combine(now, datetime.time(), tzinfo=now.tzinfo)
client.retire_partitions('async_events', 'd35a4b40-7407-40cd-a1cf-c619124ee529', begin, end)

In [ ]:
%%time
sql = "SELECT * FROM processes WHERE process_id='d35a4b40-7407-40cd-a1cf-c619124ee529'"
df = client.query(sql)
df["tsc_frequency"]

In [ ]:
%%time
#backfill day-long partitions
now = datetime.datetime.now(datetime.timezone.utc)
begin = now - datetime.timedelta(days=10)
begin = datetime.datetime.combine(begin, datetime.time(), tzinfo=begin.tzinfo)
end = datetime.datetime.combine(now, datetime.time(), tzinfo=now.tzinfo)
size = 60*60*24
client.materialize_partitions( "blocks", begin, end, size )
client.materialize_partitions( "streams", begin, end, size )
client.materialize_partitions( "processes", begin, end, size )
client.materialize_partitions( "log_entries", begin, end, size )
client.materialize_partitions( "measures", begin, end, size )
client.materialize_partitions( "log_stats", begin, end, size )


In [ ]:
%%time
#backfill one hour partitions for today
now = datetime.datetime.now(datetime.timezone.utc)
begin = datetime.datetime.combine(now, datetime.time(), tzinfo=now.tzinfo)
end = datetime.datetime.combine(now, datetime.time(now.timetz().hour), tzinfo=now.tzinfo)
size = 60*60
client.materialize_partitions( "blocks", begin, end, size )
client.materialize_partitions( "streams", begin, end, size )
client.materialize_partitions( "processes", begin, end, size )
client.materialize_partitions( "log_entries", begin, end, size )
client.materialize_partitions( "measures", begin, end, size )
client.materialize_partitions( "log_stats", begin, end, size )


In [ ]:
%%time
#backfill one minute partitions for the last hour
now = datetime.datetime.now(datetime.timezone.utc)
time = now.timetz()
begin = datetime.datetime.combine(now, datetime.time(time.hour), tzinfo=now.tzinfo)
end = datetime.datetime.combine(now, datetime.time(time.hour, time.minute), tzinfo=now.tzinfo)
size = 60
client.materialize_partitions( "blocks", begin, end, size )
client.materialize_partitions( "streams", begin, end, size )
client.materialize_partitions( "processes", begin, end, size )
client.materialize_partitions( "log_entries", begin, end, size )
client.materialize_partitions( "measures", begin, end, size )
client.materialize_partitions( "log_stats", begin, end, size )
